## Lecture 19 EEP118

General Multi Year Panel Data

Assumptions for Fixed Effect (FE) Model

Policy analysis with Panel Data



End chapter 13. Read 14.1

See R code on bcourses and also notebook in datahub for this lecture

Daily Assignment 19 posted as well as solutions

Pset 4 posted see due date in bcourses Pset 5 posted soon



Last class, with a panel of 46 cities and two years of data, 82 and 87, we estimated the following model
𝑐𝑟𝑖𝑚𝑒𝑟𝑎𝑡𝑒_𝑖𝑡= 𝛼_𝑖 + 𝛽_1  𝑢𝑛𝑒𝑚_𝑖𝑡 +𝛽_2  𝑑87 + 𝑣_𝑖𝑡


𝛽_1  = controlling for all the characteristics of the cities that do not vary over time (𝛼_𝑖  fixed effects ) and for the general effect of time common to all cities (d87) a one percent point increase in unemployment induces a 𝛽_1  increase in crime rate (number crimes per 100 people)


we used Lecture18_CRIME2 .dta   

and estimated a model

reg6n <- lm(crmrte ~ unem + d87+ +offarea+lawexpc+pcinc+factor(city) , data = mydata) 
summary(reg6n)

How did we interpret beta hat for unemployment?

Controlling for all constant characteristics of cities (that do not vary over time) =city fixed effects, and for effect of time (dummy for year 87) that captures changes in 1987 relative to 1982 factors that affect crime rate that are common to all cities,  and controlling for officer per area, law espenditures, per capita income,
a 1 percentage point increase in
 unemployment rate  is associated with a significant
 2.932 per thousand increase in crime rate 





Last lecture we had two years of data

Generalizing to many years is mechanically very easy and will open the door to many new analyses

Illustration of multi-year panel data analysis: 

# We will measure the impact of Enterprise Zones (EZ) on employment


Data: 22 cities in Indiana, from 1980 to 1988
22 cities and many years of data

Multi-year panel data:  Impact of enterprise zones on employment
Source:  data file EZUNEM (Wooldridge). 22 cities in Indiana, from 1980 to 1988


Six city enterprise zones created in 1984, and 4 more city EZ created in 1985.



In [ ]:
install.packages("pacman")
# Load the 'pacman' package
library(pacman) 
#packages to use load them now using the pacman "manager"
p_load(dplyr, haven, readr) 
#Another great feature of p_load(): if you try to load a package that is not installed on your machine, p_load() install the package for you, rather than throwing an error. For instance, let’s install and load one final package named ggplot2. 
p_load(ggplot2)


pacman::p_load(lfe, lmtest, haven, sandwich, tidyverse,stargazer)
# lfe for running fixed effects regression
# lmtest for displaying robust SE in output table
# haven for loading in dta files
# sandwich for producing robust Var-Cov matrix
# tidyverse for manipulating data and producing plots


#change into Lecture 19 directory
#setwd("/Users/sofiavillas-boas/Dropbox/EEP118_Spring2023/Lectures/Lecture19")


pacman::p_load(lfe, haven, tidyverse)




In [ ]:

#read in a Stata dataset DATA LECTURE 19
mydata <- read_dta("Lecture19_ezunem.dta")
head(mydata)

#when did city 1 get an EZ?

City 1 got EZ=1 in 1984 onwards, so in 1984

In [ ]:
# summarize data
summary(mydata)

We want to estimate the Impact of establishing Enterprise Zones (EZ) on Unemployment

The variable 𝑢𝑐𝑙𝑚𝑠_𝑖𝑡 = Unemployment claims at city i and year t

The variable 𝐸𝑍_𝑖𝑡=0 if no EZ and=1 if there is an EZ in city i at time t

uclms is number of unemployment claims file during the year in a city

year            1980 to 1988

ez              =1 if have enterprise zone, =0 o.w.

city             a city identifier from 1 to 22

What is the number of observations?    9 years times 22 cities = 198

What is the unit of observation?   ->   a city in a year

What percent of observations have EZ=1?   ---> .2323232    


the average of EZ is .2323232   

## Create a graph to let us look at evolution of Total unemployment over time



In [ ]:
# first generate sum of claims by year
#The command below creates a column in the original data with sum (over all cities) of unempl claims by year

lec19df <- mydata %>%
  group_by(year) %>%
  mutate(annual_claims = sum(uclms)) %>%
  ungroup

make graph of total (sum over all cities) umemployment claims by year

In [ ]:
# plot
ggplot(lec19df, aes(x = year, y = annual_claims)) +
  geom_line() +
  labs(title = "Unempl Claims by Year in 22 cities",
       x = "Year",
       y = "# Unemployment Claims")

## Graph Total unemployment claims (Sum of for EZ cities) and then (Sum for non EZ cities) over time


In [ ]:
# variables for graph for EZ and non EZ cities
lec19df <- mydata %>%
  group_by(city) %>%
  mutate(cityez = ifelse(max(ez)==1, "EZ", "Non-EZ")) %>%
  group_by(year, cityez) %>%
  mutate(annual_claims_ez = sum(uclms)) %>%
  ungroup()

In [ ]:
# plot for EZ and non EZ cities
ggplot(lec19df, aes(x = year, y = annual_claims_ez, group = cityez, color = cityez)) +
  geom_line() +
  scale_color_discrete(name="",
                       breaks=c("EZ", "Non-EZ"),
                       labels=c("Cities With EZ after 1984 or 1985", "Cities without EZ")) +
  labs(title = "Unemployment Claims by Year in 22 Cities",
       x = "Year",
       y = "# Unemployment Claims") 

What do you see?

1. A drop in 1982

2. 2. EZ cities always had and have lower unemployment


3. The difference between red line (cities without EZ) and blue line before 1984 had nothing to do with EZ. Since EZ only happens after 1984 for some cities

4.  The switches to EZ are in 1984 and  in 1985


5. The path of the blue line cannot be fully attributed to EZ


The empirical question is :

Does the difference in the two lines increase or decrease when cities adopt EZs?



## Regression

Lets estimate a regression model to investigate whether EZ caused an impact on unemployment


Lets consider the relationship between unemployment and EZ over time and across the 22 cities in the data using T years (T=9 here)


Model- city j  year t

 ln⁡(𝑢𝑐𝑙𝑚𝑠)_𝑗𝑡=  𝛽_1  𝐸𝑍_𝑗𝑡 + {aj } + {𝑑𝑡 } + 𝑣_𝑗𝑡


Unemployment marginal effect: (𝝏 𝒍𝒏(𝒖𝒄𝒍𝒎𝒔))/(𝑬𝒁 𝟎 𝒕𝒐 𝟏)  = 𝛽_1  percent change in unemployment claims, controlling for constant factors by city and factors common to all cities by year, with city fixed effects, and also year fixed effects, respectively.

aj captures all the characteristics of city j that are constant over time and affect unemployment -> that is, it controls for the overall level of unemployment in a city j, a city fixed effect. 

dt captures overall time patterns, year by year, that is the factors changing over time, that change year by year that affect unemployment and that are common to all cities, a year fixed effect. 


Short writing for one dummy for ALL BUT ONE city:  a2,  a3, …,a22.

And one dummy for all but ONE year (d81, d82, …dT)  exclude 1980



For city j in 1980 to 1983 no EZ (EZ=0 for all j and t<1984)

 ln⁡(𝑢𝑐𝑙𝑚𝑠)_𝑗80=𝛽_0+aj +𝑣_𝑗80          in 1980         
 
 --->  constant 𝛽_0  is 1980 (when all   other dyears=0
 
 
 ln⁡(𝑢𝑐𝑙𝑚𝑠)_𝑗81= aj +𝑑81 +𝑣_𝑗81     in 1981

For city j at time t, notation, 

ln⁡(𝑢𝑐𝑙𝑚𝑠)_𝑗𝑡= 𝛽_0+ 𝛽_1  𝐸𝑍_𝑗𝑡_ +aj +𝑑𝑡 +𝑣_𝑗𝑡





In [ ]:
#regression in levels
reg1 <- lm(uclms ~ ez, data = lec19df)
summary(reg1)


In [ ]:

#regression in log unemployment
reg0 <- lm(luclms ~ ez, data = lec19df)
summary(reg0)

We see that EZ from 0 to 1 reduced ln of log unemployment claims by -0.47 

Significantly. Or reduced unclms by 47%. Significant at 1% level. p value is p-value: 6.411e-05


## Fixed effects regression

And one dummy for all but ONE year (d81, d82, …dT)  exclude 1980

One dummy for ALL BUT ONE city:  a2,  a3, …,a22.  exclude city1


In [ ]:
#reg with fixed effects and log of unemployment claims as Y variable
reg <- lm(luclms ~ ez + factor(year) + factor(city), data = lec19df)
summary(reg)


EZ from 0 to 1 reduced log of unemployment claims by -0.10

Significantly. Or reduced unclms by 10%. Significant at 6% level.


what does the intercept estimate mean?

Ln uclms hat for city=1 and 1980

dummy for all but ONE year to take into account annual differences- exclude 1980


One dummy for ALL BUT ONE city exclude city1, to take into account city constant differences




### If we want to control for FE but not necessarily have all the estimates printed out as output, an equivalent command is in R:

In [ ]:
#xtreg equivalent using felm
reg2 <- felm(luclms ~ ez | year + city, data = lec19df)
summary(reg2)

## Make a Table

In [ ]:
#make table
#library(stargazer)
stargazer(list(reg2), type="text",keep.stat=c("n","rsq"))

#the above work in R Studio

# Panel Data Estimates of the Effects of Death Penalty Laws, and Executions on Murder Rates 1960-2000

## Another data set now: 

TODAY, as an example, “Panel Data Estimates of the Effects of Death Penalty Laws on Murder Rates 1960-2000”

Using data from FBI that I collected in March 22, 2017 

To try to perform a Replication of Donohue and Wolfers (2006) analysis



In [ ]:
#open data
mydata2 <- read_dta("dataLecture19murder.dta")
head(mydata2)
# summarize data
summary(mydata2)

#with variables to use
#by number of executions

#ex10=1 if ex10>0

#active=legal*ex10

#passive=legal*(ex10==0)


## Run regressions and Make table of results

In [ ]:
# death penalty and murders
REG1<-lm(pc_mur~legal,mydata2)
REG2<-felm(pc_mur~legal| state+year+decade_ds, data=mydata2)
REG3<-felm(pc_mur~legal+rpc_inc+ ur+ ipolice+ nonwhite +age15to19 +age20to24 | state+year+decade_ds, data=mydata2)
REG4<-felm(pc_mur~active+ passive| state+year+decade_ds, data=mydata2)
REG5<-felm(pc_mur~active+ passive+rpc_inc+ ur+ ipolice+ nonwhite +age15to19 +age20to24 | state+year+decade_ds, data=mydata2)


In [ ]:
#make table
stargazer(list(REG1,REG2,REG3, REG4, REG5), type="text",keep.stat=c("n","rsq"))


Active=1 if legal and there are executions, passive =1 if legal but no executions

 What do you conclude?



## Placebos

In [ ]:
#placebo on other murders
PREG1<-felm(pc_mur~active+ passive+rpc_inc+ ur+ ipolice+ nonwhite +age15to19 +age20to24 | state+year+decade_ds, data=mydata2)
PREG2<-felm(violentcrimerate~active+ passive+rpc_inc+ ur+ ipolice+ nonwhite +age15to19 +age20to24 | state+year+decade_ds, data=mydata2)
PREG3<-felm(pc_rob~active+ passive+rpc_inc+ ur+ ipolice+ nonwhite +age15to19 +age20to24 | state+year+decade_ds, data=mydata2)
PREG4<-felm(pc_larc~active+ passive+rpc_inc+ ur+ ipolice+ nonwhite +age15to19 +age20to24 | state+year+decade_ds, data=mydata2)
PREG5<-felm(pc_burg~active+ passive+rpc_inc+ ur+ ipolice+ nonwhite +age15to19 +age20to24 | state+year+decade_ds, data=mydata2)
PREG6<-felm(pc_auto~active+ passive+rpc_inc+ ur+ ipolice+ nonwhite +age15to19 +age20to24 | state+year+decade_ds, data=mydata2)

#make table
stargazer(list(PREG1,PREG2,PREG3, PREG4, PREG5, PREG6), type="text",keep.stat=c("n","rsq"))



Even more convincing, show that active legal death penalty has deterrence on murder but not on other crimes (e.g., auto theft) 

## Upcoming Lectures

Learn methods to approach a policy impact analysis like the soda tax

https://news.berkeley.edu/2019/04/02/berkeleys-soda-tax-election-changed-drinking-habits-months-before-prices-went-up/?fbclid=IwAR0NeswpSOdW82qoLZas348ZJpxyQLJ5oF3ZBl0j2pyrG3Olk3rvOKK1v5g


Correlation is not causation , check these crazy spurious correlations:
http://www.tylervigen.com/spurious-correlations

